# Chatbot

In [10]:
import os
import nltk
import ssl
import streamlit as st
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath("nltk_data"))
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/advik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
intents = [
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "How are you", "What's up", "Hi there", "Greetings", "Howdy", "Good to see you"],
        "responses": ["Hi there! How can I assist you today?", "Hello! What brings you here?", "Hey! Need help with something?"]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "See you later", "Goodbye", "Take care"],
        "responses": ["Goodbye", "See you later!", "Take care! Have a great day!"]
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it", "Much appreciated", "Cheers", "Thx"],
        "responses": ["You're welcome!", "No problem at all!", "Glad I could help!", "Anytime!"]
    },
    {
        "tag": "about",
        "patterns": ["What can you do", "Who are you", "What are you", "What is your purpose"],
        "responses": ["I am a chatbot created to assist you.", "My purpose is to provide helpful information and answer your questions.", "Think of me as your virtual assistant."]
    },
    {
        "tag": "help",
        "patterns": ["Help", "I need help", "Can you help me", "What should I do", "I need assistance"],
        "responses": ["Sure, what do you need help with?", "I'm here to help. Can you tell me more?", "How can I assist you today?"]
    },
    {
        "tag": "age",
        "patterns": ["How old are you", "What's your age", "When were you created"],
        "responses": ["I don't have an age. I'm a chatbot.", "I was just born in the digital world.", "Age is just a concept for me."]
    },
    {
        "tag": "weather",
        "patterns": ["What's the weather like", "How's the weather today", "Is it sunny outside"],
        "responses": ["I'm sorry, I cannot provide real-time weather information.", "Check your local weather app or website for updates.", "I currently can't access live weather data."]
    },
    {
        "tag": "budget",
        "patterns": ["How can I make a budget", "What's a good budgeting strategy", "How do I create a budget"],
        "responses": [
            "To create a budget, start by tracking your income and expenses.",
            "A good strategy is the 50/30/20 rule: 50% for needs, 30% for wants, and 20% for savings or debt.",
            "Start by setting financial goals, then allocate your income toward essentials, savings, and discretionary spending."
        ]
    },
    {
        "tag": "credit_score",
        "patterns": ["What is a credit score", "How do I check my credit score", "How can I improve my credit score"],
        "responses": [
            "A credit score is a number that represents your creditworthiness based on your credit history.",
            "You can check your credit score on websites like Credit Karma or Credit Sesame.",
            "To improve your credit score, pay bills on time, reduce debt, and avoid opening too many new accounts."
        ]
    },
    {
        "tag": "jokes",
        "patterns": ["Tell me a joke", "Make me laugh", "Got any jokes?"],
        "responses": [
            "Why don't scientists trust atoms? Because they make up everything!",
            "Why did the scarecrow win an award? He was outstanding in his field!",
            "I told my computer a joke, but it just crashed."
        ]
    },
    {
        "tag": "small_talk",
        "patterns": ["How are you?", "What's up?", "What's going on?"],
        "responses": ["I'm just a bot, but I'm here to help!", "I'm doing great, thanks for asking! How can I assist you?", "All good here! Let me know what you need."]
    },
    {
        "tag": "time_date",
        "patterns": ["What time is it?", "What's today's date?", "Tell me the time"],
        "responses": ["I'm not sure of the current time, but you can check your device!", "Sorry, I can't tell the time. Try asking your clock or phone."]
    },
    {
        "tag": "fallback",
        "patterns": [],
        "responses": ["I'm sorry, I didn't understand that.", "Can you rephrase?", "I’m not sure how to help with that."]
    }
]

In [12]:
# Create the vectorizer and classifier
vectorizer = TfidfVectorizer()  # Convert text into numerical TF-IDF features
clf = LogisticRegression(random_state=0, max_iter=10000)  # Classifier with high max_iter to ensure convergence

# Preprocess the data: Extract patterns and tags from intents
tags = []        # Stores intent tags
patterns = []    # Stores user input patterns

for intent in intents:
    patterns.extend(intent['patterns'])  # Add all patterns for the current intent
    tags.extend([intent['tag']] * len(intent['patterns']))  # Add corresponding tags (repeated)

# Train the model
x = vectorizer.fit_transform(patterns)  # Transform patterns into TF-IDF feature vectors
y = tags                                # Target labels (intent tags)
clf.fit(x, y)                           # Train the Logistic Regression classifier

print("Model training complete!")

Model training complete!


In [13]:
def chatbot(input_text):
    # Normalize input text: convert to lowercase and strip extra spaces
    input_text = input_text.lower().strip()
    
    # Transform the input into the feature vector
    input_vector = vectorizer.transform([input_text])
    
    # Predict the intent tag
    tag = clf.predict(input_vector)[0]
    
    # Check if the predicted tag exists in the intents list
    for intent in intents:
        if intent['tag'] == tag:
            # Select a random response from the matched intent
            response = random.choice(intent['responses'])
            return response
    
    # Default response if no intent is found
    return "I'm sorry, I didn't understand that. Can you please rephrase?"

In [14]:
# Assuming chatbot function and other necessary components (like vectorizer, clf, etc.) are defined

# Initialize session state for conversation history if it doesn't exist
if 'conversation' not in st.session_state:
    st.session_state['conversation'] = []

def main():
    # Streamlit app title and description
    st.title("Chatbot")
    st.write("Welcome to the chatbot. Please type a message and press Enter to start the conversation.")

    # Text input for user message
    user_input = st.text_input("You:", key="user_input")

    if user_input:
        # Get the chatbot response for the user input
        response = chatbot(user_input)
        
        # Update the conversation history
        st.session_state['conversation'].append(f"You: {user_input}")
        st.session_state['conversation'].append(f"Bot: {response}")
        
        # Display the conversation history in a scrollable area
        conversation_history = "\n".join(st.session_state['conversation'])
        st.text_area("Conversation History", value=conversation_history, height=300, max_chars=None, key="conversation_history", disabled=True)

        # Stop if the user says goodbye
        if response.lower() in ['goodbye', 'bye']:
            st.session_state['conversation'].append("Bot: Thank you for chatting with me. Have a great day!")
            st.text_area("Conversation History", value="\n".join(st.session_state['conversation']), height=300, max_chars=None, key="final_conversation", disabled=True)
            st.stop()

if __name__ == '__main__':
    main()